In [1]:
import itertools
import random, math
from collections import OrderedDict, Counter
import re
import nltk
from nltk.corpus import brown, gutenberg
from nltk.probability import FreqDist
from nltk.corpus import stopwords

# Corpus

In [3]:
gutenberg.fileids()[3]

'bible-kjv.txt'

In [4]:
samples = gutenberg.sents(gutenberg.fileids()[3])
pattern = re.compile("[A-Za-z]+")
stop_w =  set(stopwords.words('english'))
corpus = []
for sent in samples:
    sent = [w.lower() for w in sent]
    sent = [w for w in sent if w not in stop_w]
    sent = [w.replace('\n', ' ') for w in sent]
    sent = [w for w in sent if pattern.fullmatch(w)]
    if len(sent) > 5:
        corpus.append(sent)

In [5]:
fre_dist = FreqDist()
for sent in corpus:
    fre_dist.update(sent)
fre_dist = {k : v for k, v in fre_dist.items() if v > 5}

In [6]:
vocab_size = len(fre_dist)
idx_to_word = {idx: word for idx,  word in enumerate(fre_dist.keys())}
word_to_idx = {word: idx for idx, word in idx_to_word.items()}


## convert word to index

In [7]:
corpus_indexed = [[word_to_idx[word] for word in sent if word in word_to_idx]for sent in corpus]
corpus_indexed = [sent for sent in corpus_indexed if len(sent) > 5]
fre_dist_indexed = {word_to_idx[w]: f for w, f in fre_dist.items()}

# Dataset

In [8]:
import torch
import numpy as np
import torch.functional as F
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import tqdm

In [9]:
class NegativeSampler:
    def __init__(self, corpus, sample_ratio=0.75):
        self.sample_ratio = sample_ratio
        self.sample_table =  self.__build_sample_table(corpus)
        self.table_size = len(self.sample_table)
        
    def __build_sample_table(self, corpus):
        counter = dict(Counter(list(itertools.chain.from_iterable(corpus))))
        words = np.array(list(counter.keys()))
        probs = np.power(np.array(list(counter.values())), self.sample_ratio)
        normalizing_factor = probs.sum()
        probs = np.divide(probs, normalizing_factor)
        
        sample_table = []

        table_size = 1e8
        word_share_list = np.round(probs * table_size)
        '''
         the higher prob, the more shares in  sample_table
        '''
        for w_idx, w_fre in enumerate(word_share_list):
            sample_table += [words[w_idx]] * int(w_fre)

#         sample_table = np.array(sample_table) // too slow
        return sample_table
    
    def generate(self, sample_size=6):

        negatvie_samples = [self.sample_table[idx] for idx in np.random.randint(0, self.table_size, sample_size)]
        return np.array(negatvie_samples)
    

In [10]:
class SkipGramWithNGEDataset(torch.utils.data.Dataset):
    def __init__(self, corpus, window_size=5, sentence_length_threshold=5, negative_sample_size=10):
        self.window_size = window_size
        self.sentence_length_threshold = sentence_length_threshold
        self.negative_sample_size = negative_sample_size
        
        self.corpus = self.__subsampling_frequenct_words(corpus)
        self.pairs = self.__generate_pairs(self.corpus, window_size)
        self.negative_sampler = NegativeSampler(self.corpus)
        
    def __sub_sample(self, x, alpha=3):
        pow_ = math.pow(10, alpha)
        s = math.sqrt(x * pow_)
        return (s + 1) / (x * pow_)
    
    def __subsampling_frequenct_words(self, corpus):
        counter = dict(Counter(list(itertools.chain.from_iterable(corpus))))
        sum_word_count = sum(list(counter.values()))
        
        word_ratio ={w: count / sum_word_count  for w, count in counter.items()}
        
        word_subsample_frequency = {k: self.__sub_sample(v) for k, v in word_ratio.items()}
        
        filtered_corpus = [] 
        for sent in corpus:
            filtered_sent = []
            for w in sent:
                if random.random() < word_subsample_frequency[w]:
                      filtered_sent.append(w)
            filtered_corpus.append(filtered_sent)
        return filtered_corpus
    
    
        
    def __generate_pairs(self, corpus, windows_size):     
        pairs = []
        for sentence in corpus:
            if len(sentence) < self.sentence_length_threshold:
                continue

            for center_word_pos in range(len(sentence)):
                for shift in range(-windows_size, windows_size + 1):
                    context_word_pos = center_word_pos + shift
                    
                    if (0 <= context_word_pos < len(sentence)) and context_word_pos != center_word_pos:
                        pairs.append((sentence[center_word_pos], sentence[context_word_pos]))
        return pairs  # [(centerword, a_context_word)]

    def __len__(self):
        return len(self.pairs)
    
    '''
        @return:  1 center_w, 1 context_w, n negative sample words
    '''
    def __getitem__(self, index):
        center_w, context_w = self.pairs[index]

        return np.array([center_w]), np.array([context_w]), self.negative_sampler.generate(self.negative_sample_size)

# Skipgram with negative sampling
$$
\mathcal{L}_\theta = - [ \log \sigma(\text{v}^\top_{w_I} \text{v}'_{w_{O,j}}) +  \sum^M_{\substack{i=1 \\ \tilde{w}_i \sim Q}}\exp(\text{v}^\top_{w_I} \text{v}'_{\tilde{w}_i})]
$$

In [11]:
class SkipGramNEG(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.syn0 = nn.Embedding(vocab_size, embedding_dim) # |V| x |K|
        self.neg_syn1 = nn.Embedding(vocab_size, embedding_dim) # |V| x |K|
        torch.nn.init.constant_(self.neg_syn1.weight.data, val=0)
        
    def forward(self, center: torch.Tensor, context: torch.Tensor, negative_samples: torch.Tensor):
        # center : [b_size, 1]
        # context: [b_size, 1]
        # negative_sample: [b_size, negative_sample_num]
        embd_center = self.syn0(center)  # [b_size, 1, embedding_dim]
        embd_context = self.neg_syn1(context) # [b_size, 1, embedding_dim]
        embd_negative_sample = self.neg_syn1(negative_samples) # [b_size, negative_sample_num, embedding_dim]
        
        prod_p =  (embd_center * embd_context).sum(dim=1).squeeze()  # [b_size]
        loss_p =  F.logsigmoid(prod_p).mean() # 1
        
        
        prod_n = (embd_center * embd_negative_sample).sum(dim=2) # [b_size, negative_sample_num]
        loss_n = F.logsigmoid(-prod_n).sum(dim=1).mean() # 1
        return -(loss_p + loss_n)
        

## training stage

In [12]:
EMBEDDING_DIM = 100
model = SkipGramNEG(vocab_size, EMBEDDING_DIM)
optimizer = optim.Adam(model.parameters(), lr=0.001,  weight_decay=1e-6)


In [13]:
dataset = SkipGramWithNGEDataset(corpus_indexed)
data_loader = DataLoader(dataset, batch_size=500, num_workers=0)

In [14]:
log_interval = 100
for epoch_i in range(2):
    total_loss = 0
    model.train()
    tk0 = tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0)
    for i, (center, context, neg_samples) in enumerate(tk0):
        loss = model(center, context, neg_samples)
        model.zero_grad()
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        if(i + 1) % log_interval == 0:
            tk0.set_postfix(loss = total_loss/log_interval)
            total_loss = 0

100%|██████████| 4205/4205 [03:51<00:00, 18.18it/s, loss=0.757]


## fetch word embedding

In [15]:
syn0 = model.syn0.weight.data
neg_syn1 = model.neg_syn1.weight.data

w2v_embedding = (syn0 + neg_syn1) / 2
w2v_embedding = w2v_embedding.numpy()
l2norm = np.linalg.norm(w2v_embedding, 2, axis=1, keepdims=True)
w2v_embedding = w2v_embedding / l2norm


# Evaluate

In [16]:
from sklearn.neighbors import NearestNeighbors
class NNeighbors:
    def __init__(self, NN_model, word_embedding, idx_to_word_dict, word_to_idx_dict):
        self.NN_model = NN_model
        self.word_embedding = word_embedding
        self.idx_to_word_dict = idx_to_word_dict
        self.word_to_idx_dict = word_to_idx_dict
    
    def get_neighbors(self, word):
        idx = self.word_to_idx_dict[word]
        embedding = self.word_embedding[[idx]]
        dists, neighbors = self.NN_model.kneighbors(embedding)
        dists, neighbors = dists[0], neighbors[0]
        
        pairs = OrderedDict()
        for n_idx, d in zip(neighbors, dists):
            pairs[self.idx_to_word_dict[n_idx]] = d
        return pairs
        
        
    

In [17]:
neighbor = NearestNeighbors(n_neighbors=10, algorithm='auto', p=2).fit(w2v_embedding)
nn_neighbors = NNeighbors(neighbor, w2v_embedding, idx_to_word, word_to_idx)

In [18]:
nn_neighbors.get_neighbors('jesus')

OrderedDict([('jesus', 0.0),
             ('christ', 0.7522091979029454),
             ('gospel', 0.9632666194513442),
             ('peter', 1.1310211737966784),
             ('disciples', 1.1436218856215599),
             ('church', 1.2036696221080758),
             ('passed', 1.2268319584910587),
             ('noise', 1.2347588831506333),
             ('preach', 1.2372968309935133),
             ('send', 1.2379122748838614)])

## cosine similarity

In [19]:
class CosineSimilarity:
    def __init__(self, word_embedding, idx_to_word_dict, word_to_idx_dict):
        self.word_embedding = word_embedding # normed already
        self.idx_to_word_dict = idx_to_word_dict
        self.word_to_idx_dict = word_to_idx_dict
        
    def get_synonym(self, word, topK=10):
        idx = self.word_to_idx_dict[word]
        embed = self.word_embedding[idx]
        
        cos_similairty = w2v_embedding @ embed
        
        topK_index = np.argsort(-cos_similairty)[:topK]
        pairs = []
        for i in topK_index:
            w = self.idx_to_word_dict[i]
            pairs.append((w, cos_similairty[i]))
        return pairs
        

In [20]:
cosinSim = CosineSimilarity(w2v_embedding, idx_to_word, word_to_idx)
cosinSim.get_synonym('christ')

[('christ', 1.0),
 ('jesus', 0.7170907),
 ('gospel', 0.4621805),
 ('peter', 0.39412546),
 ('disciples', 0.3873747),
 ('noise', 0.28152165),
 ('asleep', 0.26372147),
 ('taught', 0.2422184),
 ('zarhites', 0.24168596),
 ('nobles', 0.23950878)]

In [21]:
cosinSim.get_synonym('jesus')

[('jesus', 1.0),
 ('christ', 0.7170907),
 ('gospel', 0.5360588),
 ('peter', 0.3603956),
 ('disciples', 0.3460646),
 ('church', 0.2755898),
 ('passed', 0.24744174),
 ('noise', 0.23768528),
 ('preach', 0.23454829),
 ('send', 0.2337867)]